In [98]:
###############################
## This script links the flood sum map that was generated in GEE to the corresponding flood frequenices 
## for the Prek Area 
## exüprted from this script: https://code.earthengine.google.com/f69af6afbec2c0a5d43a173a965c67be

In [99]:
# import necessary packages
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import datetime
import os
import matplotlib.pyplot as plt
from io import StringIO
import matplotlib.dates as mdates
from matplotlib.dates import DateFormatter
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
import math
from matplotlib.dates import date2num
from pandas import DataFrame

In [100]:
# water level at Koh Kehl 

wl_long = pd.read_csv('wl_KK.csv')
wl_long['date'] = pd.to_datetime(wl_long['date'])
wl_long['date'] = wl_long['date'].dt.strftime('%Y-%m-%d')
wl_long.columns = ['huh', 'date', 'wl_long']
wl_long = wl_long.drop(columns=['huh'])
wl_long = wl_long.set_index('date')
wl_long['wl_long'] = wl_long['wl_long']-1 # <------ -1 correct for elevation 

# for data availability comparison:
wl_long.index = pd.to_datetime(wl_long.index)
wl_long1 = wl_long.resample('d').mean()

wl_KK_daily = wl_long1  
wl_KK_daily.columns = ['wl_KK']
wl_original = wl_KK_daily

#wl_KK_daily = wl_KK_daily.rolling(center=False, window=3, min_periods=1).mean().round(2)

wl_KK_daily.to_csv('wl_KK_smoothed.csv')

wl_KK_daily.head()

,wl_KK
date,
1990-09-05,7.16
1990-09-06,7.20
1990-09-07,7.22
1990-09-08,7.22
1990-09-09,7.21


In [134]:
### add data and functions to calculate delay times 

# Prek Area: calculate the delay times according to water levels for each image 

delay_PA = pd.read_csv('Inundations_WaterLevels_PA.csv', sep=';', decimal=',').round(2)
delay_PA.head(3)

mean_delay_PA = delay_PA['DelayTime'].mean()
#print(mean_delay_PA)

# Zone 2C: calculate the delay times according to water levels for each image 

delay_2C = pd.read_csv('Inundations_WaterLevels_2C.csv', sep=';', decimal=',').round(2)
delay_2C.head(10)

mean_delay_2C = delay_2C['DelayTime'].mean()
#print(mean_delay_2C)

# for each water level in KK, find the date that matches it, according to the table  - then check, if there is a Sentinel image for that day in the collection

def find_delay_PA(wl):
    test_value = wl
    df_temp = delay_PA.iloc[(delay_PA['WL']-test_value).abs().argsort()[:1]] #<--- match wl
    time_days = int(df_temp['DelayTime'].round(0)) #<- select corresponding delay
    return time_days

def find_delay_2C(wl):
    test_value = wl
    df_temp = delay_2C.iloc[(delay_2C['WL']-test_value).abs().argsort()[:1]] #<--- match wl
    time_days = int(df_temp['DelayTime'].round(0)) #<- select corresponding delay
    return time_days

## add the individual inundation delays for each wl value at KK - convert to list of dates 

wl_KK_daily['date'] = wl_KK_daily.index
wl_KK_daily['delay_PA'] = wl_KK_daily['wl_KK'].apply(find_delay_PA)
wl_KK_daily['delay_2C'] = wl_KK_daily['wl_KK'].apply(find_delay_2C)
wl_KK_daily['delay_PA'] = pd.to_timedelta(wl_KK_daily['delay_PA'], unit='d')
wl_KK_daily['Sentinel_date_PA'] = wl_KK_daily['date']+ wl_KK_daily['delay_PA']
wl_KK_daily['delay_2C'] = pd.to_timedelta(wl_KK_daily['delay_2C'], unit='d')
wl_KK_daily['Sentinel_date_2C'] = wl_KK_daily['date']+ wl_KK_daily['delay_2C']
wl_KK_daily = wl_KK_daily.drop(columns=['delay_PA'])
wl_KK_daily = wl_KK_daily.drop(columns=['delay_2C'])

#wl_KK_daily['new_date'] = (wl_KK_daily['date'] + datetime.timedelta(days=wl_KK_daily['delay']))

wl_KK_daily.tail(10)

,wl_KK,date,Sentinel_date_PA,Sentinel_date_2C
date,,,,
2021-03-13,0.920,2021-03-13,2021-03-27,2021-04-07
2021-03-14,0.970,2021-03-14,2021-03-23,2021-04-07
2021-03-15,0.980,2021-03-15,2021-03-24,2021-04-08
2021-03-16,1.000,2021-03-16,2021-03-25,2021-04-09
2021-03-17,1.030,2021-03-17,2021-03-26,2021-04-10
2021-03-18,0.985,2021-03-18,2021-03-27,2021-04-11
2021-03-19,0.995,2021-03-19,2021-03-28,2021-04-12
2021-03-20,0.995,2021-03-20,2021-03-29,2021-04-13
2021-03-21,0.920,2021-03-21,2021-04-04,2021-04-15


In [136]:
wl_KK_daily[(wl_KK_daily['Sentinel_date_2C'] == '2015-09-12')]

,wl_KK,date,Sentinel_date_PA,Sentinel_date_2C
date,,,,
2015-08-19,4.58,2015-08-19,2015-08-31,2015-09-12


In [58]:
##### import list of dates manually generated 

dates_simple = pd.read_csv('IE_PA_Simple.csv')
dates_simple['Date'] = pd.to_datetime(dates_simple['Date'])
dates_simple['Date'] = dates_simple['Date'].dt.strftime('%Y-%m-%d')
dates_simple['date'] = dates_simple['Date']

dates_simple = dates_simple.set_index('date')

dates_simple.head(30)

,Date
date,
2019-07-15,2019-07-15
2018-07-15,2018-07-15
2017-07-15,2017-07-15
2016-07-15,2016-07-15
2019-08-01,2019-08-01
2018-08-01,2018-08-01
2017-08-01,2017-08-01
2016-08-01,2016-08-01
2019-08-15,2019-08-15


In [57]:
##### import list of dates from GEE 

dates_GEE = pd.read_csv('Datesofimages_PA.csv')
dates_GEE = dates_GEE.drop(columns=['system:index', '.geo'])
dates_GEE.columns = ['list']

#dates_GEE = dates_GEE['list']

dates_GEE = dates_GEE['list'].iloc[-1]

dates = StringIO(dates_GEE)

df = pd.read_csv(dates, sep=",").T

df['date'] = df.index

#df.columns = ['date']

df = df.reset_index()

df = df.drop(columns=['index'])

df['date'] = df['date'].map(lambda x: x.lstrip('[').rstrip(']'))

df['date'] = pd.to_datetime(df['date']).dt.strftime('%Y-%m-%d')


df.head()

,date
0,2000-10-20
1,2011-09-17
2,2017-05-17
3,2017-05-29
4,2017-06-10


In [8]:
##### match them to the water levels at KK at the correct date

wl_KK_PA = wl_KK_daily.set_index('Sentinel_date_PA')

flood_link = dates_simple.merge(wl_KK_PA, how='inner', left_index=True, right_index=True)

flood_link = flood_link.drop(columns=['Date', 'Sentinel_date_2C'])

flood_link = flood_link.sort_values('wl_KK')

flood_link['date_true'] = flood_link.index

flood_link.index = np.arange(1, len(flood_link) + 1)

flood_link['count_reached'] = flood_link.index

#flood_link.head()

In [9]:
#### for each value in colum: count how often it has been reached by a certain date in the years since 1990 overall 

wl_long = pd.read_csv('wl_KK.csv')
wl_long['date'] = pd.to_datetime(wl_long['date'])
wl_long['date'] = wl_long['date'].dt.strftime('%Y-%m-%d')
wl_long.columns = ['huh', 'date', 'wl_long']
wl_long = wl_long.drop(columns=['huh'])
wl_long = wl_long.set_index('date')
wl_long['wl_long'] = wl_long['wl_long']#-1 # <------ -1 correct for elevation 

# for data availability comparison:
wl_long.index = pd.to_datetime(wl_long.index)
wl_original = wl_long.resample('d').mean()
wl_original['date'] = wl_original.index
wl_original['monthday'] = wl_original['date'].dt.strftime('%m-%d')
wl_original['year'] = wl_original['date'].dt.strftime('%Y')

wl_original.head()


years = wl_original.year.unique()

def counting_times_reached(wl):
        list1 = []
        for year in years: 
            year = str(year)
            wl_year = wl_original[(wl_original['year']==year)] # filter for year 
            wl_decide = wl_year[(wl_year['monthday']<=testdate)] # filter before date
            #wl_min = wl_decide1['wl_long'].min()
            #date_min = wl_decide1[(wl_decide1.wl_long==wl_min).idxmax()]['date']  
            #wl_decide = wl_decide1[(wl_decide1['date'] > date_min)] # after dry season min
            check = (wl_decide['wl_long'] >= wl).any()#.astype(int) # check if present
            list1.append(check)
        total_sum = float(sum(list1))/float(len(years))
        return total_sum
            
            

In [10]:
testdate = '11-31'
flood_link['freq_overall'] = (flood_link['wl_KK'].apply(counting_times_reached))#/number_years

testdate = '07-15'
flood_link['freq_Jul15'] = (flood_link['wl_KK'].apply(counting_times_reached))#/number_years

testdate = '07-31'
flood_link['freq_Jul31'] = (flood_link['wl_KK'].apply(counting_times_reached))#/number_years 

testdate = '08-15'
flood_link['freq_Aug15'] = (flood_link['wl_KK'].apply(counting_times_reached))#/number_years 

testdate = '08-31'
flood_link['freq_Aug31'] = (flood_link['wl_KK'].apply(counting_times_reached))#/number_years 

testdate = '09-15'
flood_link['freq_Sep25'] = (flood_link['wl_KK'].apply(counting_times_reached))#/number_years 


##### convert to 


In [11]:
flood_link.tail()

,wl_KK,date,date_true,count_reached,freq_overall,freq_Jul15,freq_Jul31,freq_Aug15,freq_Aug31,freq_Sep25
26,7.525,2019-09-08,2019-09-25,26,0.43750,0.0,0.0,0.0625,0.12500,0.28125
27,7.660,2000-10-03,2000-10-20,27,0.31250,0.0,0.0,0.0625,0.09375,0.18750
28,7.675,2018-08-14,2018-08-31,28,0.28125,0.0,0.0,0.0625,0.09375,0.15625
29,7.755,2018-09-08,2018-09-25,29,0.18750,0.0,0.0,0.0000,0.03125,0.09375
30,7.780,2018-08-29,2018-09-15,30,0.15625,0.0,0.0,0.0000,0.03125,0.09375
